In [1]:
# Import packages and libraries
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras import layers

In [3]:
# Load training data into dataframe
X_train = pd.read_csv('../Data/Training_Features.csv')
X_train.drop(columns=['date_recorded', 'permit', 'public_meeting'], inplace=True)

y_train = pd.read_csv('../Data/Training_Labels.csv')
y_train = y_train['status_group']

In [ ]:
train, val, test = np.split(dat.sample(frac=1), [int(0.8*len(dataframe)), int(0.9*len(dataframe))])